<h3>As per usual we will use the most user friendly way of doing Machine Learning and Data Science</h3>

<h3>Since this problem is generally well known, we will try to apply best known practices. Also we will try not to use the heavy machinery.</h3>


In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn #maybe this one is an overkill since we will only use some of the functions
import datetime

In [2]:
import os
dir_path = os.path.dirname(os.path.realpath('Starting Notebook for Assigment.ipynb'))
print(dir_path)

C:\Users\Faris Hambo\Things Solver


In [4]:
df = pd.read_excel(dir_path+'\\base_dataset.xlsx',engine='openpyxl')

In [5]:
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [6]:
df.describe()

,Quantity,UnitPrice,CustomerID
count,541909.000000,541909.000000,406829.000000
mean,9.552250,4.611114,15287.690570
std,218.081158,96.759853,1713.600303
min,-80995.000000,-11062.060000,12346.000000
25%,1.000000,1.250000,13953.000000
50%,3.000000,2.080000,15152.000000
75%,10.000000,4.130000,16791.000000
max,80995.000000,38970.000000,18287.000000


<h3>Now we will introduce necessary function that we will use to expand the existing dataset or do some data wrangling in general</h3>

<h3>For start let's create lambda function that calculates money spent for each record</h3>

In [7]:
def amountPerRecord(record):
    return record['Quantity']*record['UnitPrice']

In [8]:
df = df.assign(AmountSpent = lambda x: amountPerRecord(x))


In [9]:
df.InvoiceDate = df.InvoiceDate.dt.date


In [10]:
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,AmountSpent
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01,2.55,17850.0,United Kingdom,15.30
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01,3.39,17850.0,United Kingdom,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01,2.75,17850.0,United Kingdom,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01,3.39,17850.0,United Kingdom,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01,3.39,17850.0,United Kingdom,20.34


<h3> While we are on subject of adding or droping columns, let's also some basic feature engineering aswell. It's pretty obvious that this is some form of shop (whatever that may be). So if we wanted to see how a certain item was sold, we probably wouldn't need it's description. Code of Stock will suffice in general, especially for us developers. So let's assume that description isn't really necessary at this point. Also we changed the dataty </h3>

In [11]:
df.drop(['Description'], axis=1, inplace = True)

In [12]:
df.head()

,InvoiceNo,StockCode,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,AmountSpent
0,536365,85123A,6,2010-12-01,2.55,17850.0,United Kingdom,15.30
1,536365,71053,6,2010-12-01,3.39,17850.0,United Kingdom,20.34
2,536365,84406B,8,2010-12-01,2.75,17850.0,United Kingdom,22.00
3,536365,84029G,6,2010-12-01,3.39,17850.0,United Kingdom,20.34
4,536365,84029E,6,2010-12-01,3.39,17850.0,United Kingdom,20.34


<h3>Now we will try to answer first part of the question, or in other word let's find RFMT.</h3>

In [13]:
def frequency (id):
    
    tempDf = df.loc[df['CustomerID']==id] #this way we extract that customer that is part of our interest
    
    visited = set()
    
    for i in range(len(tempDf)):
        visited.add(tempDf['InvoiceNo'].iloc[i])
        
    return len(visited)

In [14]:
print(frequency(17850))


35


In [15]:
def monetary_value(id):
    
    tempDf = df.loc[df['CustomerID']==id]
    totalMoney = 0
    
    for i in range(len(tempDf)):
        totalMoney += tempDf['AmountSpent'].iloc[i]
        
    return totalMoney


In [16]:
print(monetary_value(17850))

5288.630000000009


In [17]:
def lifetime(id):
    tempDf = df.loc[df['CustomerID']==id]
    tempDf.sort_values(by=['InvoiceDate'])
    return (datetime.date(2021, 8, 12)-tempDf['InvoiceDate'].iloc[0]).days

In [18]:
lifetime(17850)

3907

In [19]:
def recency(id: int) -> int:
    tempDf = df.loc[df['CustomerID']==id]
    tempDf.sort_values(by=['InvoiceDate'])
    return (datetime.date(2021, 8, 12)-tempDf['InvoiceDate'].iloc[-1]).days
    

In [38]:
uniqueIds = df[["CustomerID"]].drop_duplicates()
uniqueIds.dropna(inplace = True)


In [40]:
new_dict = {'Id':[],'Frequency':[],'Recency':[],'Lifetime':[],'MonetaryValue':[]}

for i in range(len(uniqueIds)):

    new_dict['Id'].append(uniqueIds['CustomerID'].iloc[i])
    new_dict['Frequency'].append(frequency(uniqueIds['CustomerID'].iloc[i]))
    new_dict['MonetaryValue'].append(monetary_value(uniqueIds['CustomerID'].iloc[i]))
    new_dict['Lifetime'].append(lifetime(uniqueIds['CustomerID'].iloc[i]))
    new_dict['Recency'].append(recency(uniqueIds['CustomerID'].iloc[i]))
    


In [42]:
usersDf = pd.DataFrame.from_dict(new_dict)
usersDf.head()

,Id,Frequency,Recency,Lifetime,MonetaryValue
0,17850.0,35,3836,3907,5288.63
1,13047.0,18,3565,3907,3079.10
2,12583.0,18,3536,3907,7187.34
3,13748.0,5,3629,3907,948.25
4,15100.0,6,3864,3907,635.10


In [43]:
usersDf.to_csv('dataUserLevel.csv', encoding='utf-8', index=False)